In [1]:
%cd DA_DFD

[Errno 2] No such file or directory: 'DA_DFD'
/data/home/jkataok1/DA_DFD


In [ ]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
from glob import glob
import os
import scipy.io
from sklearn.linear_model import LogisticRegression
from scipy.signal import spectrogram
import torch.nn as nn
import torch
from collections import OrderedDict
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import torch.utils.data as Data
import wandb

sys.path.append("/data/home/jkataok1/DA_DFD")

from src.dataloader import generate_dataset
from src.ast_models import ASTModel
from src.helper import count_batch_on_large_dataset

/data/home/jkataok1/mlenv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = ASTModel(label_dim=4, input_tdim=65, input_fdim=18, imagenet_pretrain=False, audioset_pretrain=False)
checkpoint = "/data/home/jkataok1/DA_DFD/src_models/CWRU_all_spectrogram_IMS_0_spectrogram_src_ast.pth"

state_dict_temp = torch.load(checkpoint)
model_params = model.state_dict()
for name, param in state_dict_temp.items():
    if name in model_params:
        model_params[name].copy_(param)
        print("copy {}".format(name), sep="\r")
    else:
        print("skip {}".format(name), sep="\r")
model.load_state_dict(state_dict_temp, strict=True)

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        # add mlp projection head
        embedding_size = 1024

        mlp_dim= model.mlp_head[-1].in_features
        self.projection = nn.Sequential(
           nn.Linear(in_features=mlp_dim, out_features=mlp_dim),
           nn.BatchNorm1d(mlp_dim),
           nn.ReLU(),
           nn.Linear(in_features=mlp_dim, out_features=embedding_size),
           nn.BatchNorm1d(embedding_size))

    def forward(self, x):
        x = self.projection(x)

        return x

model.mlp_head=Identity()
model = model.to("cuda")

---------------AST Model Summary---------------
ImageNet pretraining: False, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=5
copy v.cls_token
copy v.pos_embed
copy v.dist_token
copy v.patch_embed.proj.weight
copy v.patch_embed.proj.bias
copy v.blocks.0.norm1.weight
copy v.blocks.0.norm1.bias
copy v.blocks.0.attn.qkv.weight
copy v.blocks.0.attn.qkv.bias
copy v.blocks.0.attn.proj.weight
copy v.blocks.0.attn.proj.bias
copy v.blocks.0.norm2.weight
copy v.blocks.0.norm2.bias
copy v.blocks.0.mlp.fc1.weight
copy v.blocks.0.mlp.fc1.bias
copy v.blocks.0.mlp.fc2.weight
copy v.blocks.0.mlp.fc2.bias
copy v.blocks.1.norm1.weight
copy v.blocks.1.norm1.bias
copy v.blocks.1.attn.qkv.weight
copy v.blocks.1.attn.qkv.bias
copy v.blocks.1.attn.proj.weight
copy v.blocks.1.attn.proj.bias
copy v.blocks.1.norm2.weight
copy v.blocks.1.norm2.bias
copy v.blocks.1.mlp.fc1.weight
copy v.blocks.1.mlp.fc1.bias
copy v.blocks.1.mlp.fc2.weight
copy v.blocks.1.mlp.fc2.bias
copy v.bloc

In [4]:

class ContrastiveLoss(nn.Module):
   """
   Vanilla Contrastive loss, also called InfoNceLoss as in SimCLR paper
   """
   def __init__(self, batch_size, temperature=0.5):
       super().__init__()
       self.batch_size = batch_size
       self.temperature = temperature
       self.mask = (~torch.eye(batch_size * 2, batch_size * 2, dtype=bool)).float()

   def device_as(self, t1, t2):
        """
        Moves t1 to the device of t2
        """
        return t1.to(t2.device)

   def calc_similarity_batch(self, a, b):
       representations = torch.cat([a, b], dim=0)
       return F.cosine_similarity(representations.unsqueeze(1), representations.unsqueeze(0), dim=2)

   def forward(self, proj_1, proj_2):
       """
       proj_1 and proj_2 are batched embeddings [batch, embedding_dim]
       where corresponding indices are pairs
       z_i, z_j in the SimCLR paper
       """
       batch_size = proj_1.shape[0]
       z_i = F.normalize(proj_1, p=2, dim=1)
       z_j = F.normalize(proj_2, p=2, dim=1)

       similarity_matrix = self.calc_similarity_batch(z_i, z_j)

       sim_ij = torch.diag(similarity_matrix, batch_size)
       sim_ji = torch.diag(similarity_matrix, -batch_size)

       positives = torch.cat([sim_ij, sim_ji], dim=0)

       nominator = torch.exp(positives / self.temperature)

       denominator = self.device_as(self.mask, similarity_matrix) * torch.exp(similarity_matrix / self.temperature)

       all_losses = -torch.log(nominator / torch.sum(denominator, dim=1))
       loss = torch.sum(all_losses) / (2 * self.batch_size)
       return loss

In [5]:
class Augment:
   """
   A stochastic data augmentation module
   Transforms any given data example randomly
   resulting in two correlated views of the same example,
   denoted x ̃i and x ̃j, which we consider as a positive pair.
   """

   def __init__(self, img_size, s=1):
       color_jitter = T.ColorJitter(
           0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s
       )
       # 10% of the image
       blur = T.GaussianBlur(1, (0.1, 2.0))

       self.train_transform = torch.nn.Sequential(
           T.RandomResizedCrop(size=img_size),
           T.RandomHorizontalFlip(p=0.5),  # with 0.5 probability
           T.RandomApply([color_jitter], p=0.8),
           T.RandomApply([blur], p=0.5),
           #T.RandomGrayscale(p=0.2),
           # imagenet stats
           #T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
       )

   def __call__(self, x):
       return self.train_transform(x), self.train_transform(x)

In [6]:
def define_param_groups(model, weight_decay, optimizer_name):
   def exclude_from_wd_and_adaptation(name):
       if 'bn' in name:
           return True
       if optimizer_name == 'lars' and 'bias' in name:
           return True

   param_groups = [
       {
           'params': [p for name, p in model.named_parameters() if not exclude_from_wd_and_adaptation(name)],
           'weight_decay': weight_decay,
           'layer_adaptation': True,
       },
       {
           'params': [p for name, p in model.named_parameters() if exclude_from_wd_and_adaptation(name)],
           'weight_decay': 0.,
           'layer_adaptation': False,
       },
   ]
   return param_groups

In [13]:
learning_rate = 0.0001
weight_decay = 1e-4
params= define_param_groups(model, weight_decay=weight_decay, optimizer_name="Adam")
optimizer = torch.optim.Adam(params, lr=learning_rate)
batch_size = 128*2  # Set the batch size that fits into your GPU
epochs = 200
temperature = 0.5
loss = ContrastiveLoss(batch_size, temperature=temperature)

src_dataset, tar_dataset = generate_dataset("/data/home/jkataok1/DA_DFD/data/processed",
                    "CWRU", "IMS", "all_spectrogram", "0_spectrogram")
src_dataloader = Data.DataLoader(src_dataset, 
                                    batch_size=batch_size, shuffle=True, drop_last=True)
tar_dataloader = Data.DataLoader(tar_dataset,
                                    batch_size=batch_size, shuffle=True, drop_last=True) 

hyperparameter_defaults = dict(
    epochs=epochs,
    batch_size=batch_size,
    lr=learning_rate,
    weight_decay=weight_decay,
)
log = "/data/home/jkataok1/DA_DFD/log/SimCLR_01"
if not os.path.isdir(log):
    os.makedirs(log)
    print("create new log directory")   
model_name = "SimCLR.pth"
wandb.init(config=hyperparameter_defaults, name=log, project="DA_DFD")

loss,▅▅█▆▁▇█▇█▇
loss_src,▂▂▂▁▁▄█▄▅▄
loss,2.38381
loss_src,6.07791
loss_tar,None.Tensor.item


In [12]:
batch_count = count_batch_on_large_dataset(src_dataloader, tar_dataloader)
# count total iteration
num_itern_total = epochs * batch_count
# initialize epoch
epoch = 0
count_itern_each_epoch = 0
best_acc = 0.0
accum_iter = 5
augmentation = Augment(img_size=(65, 18))
best_loss = 1e+10
model.train()

for itern in range(num_itern_total):
    src_train_batch = enumerate(src_dataloader)
    tar_train_batch = enumerate(tar_dataloader)

    if (itern==0 or count_itern_each_epoch==batch_count):
        if itern != 0:
            count_itern_each_epoch = 0
            epoch += 1
    
    try:
        (src_idx, src_input, src_target) = src_train_batch.__next__()[1]
    except StopIteration:
        src_train_batch = enumerate(src_dataloader)
        (src_idx, src_input, src_target) = src_train_batch.__next__()[1]

    try:
        (tar_idx, tar_input, tar_target) = tar_train_batch.__next__()[1]
    except StopIteration:
        tar_train_batch = enumerate(tar_dataloader)
        (tar_idx, tar_input, tar_target) = tar_train_batch.__next__()[1]

    src_input_extend = src_input.unsqueeze(1).cuda()
    tar_input_extend = tar_input.unsqueeze(1).cuda()

    src_input_1, src_input_2 = augmentation(src_input_extend)
    tar_input_1, tar_input_2 = augmentation(tar_input_extend)  

    src_z_1 = model(src_input_1.squeeze(1))
    src_z_2 = model(src_input_2.squeeze(1))
    loss_src = loss(src_z_1, src_z_2)

    tar_z_1 = model(tar_input_1.squeeze(1))
    tar_z_2 = model(tar_input_2.squeeze(1))
    loss_tar = loss(tar_z_1, tar_z_2)
    loss_all = (loss_src + loss_tar) / accum_iter
    wandb.log({"loss": loss_all.item(),
               "loss_src": loss_src.item(),
               "loss_tar": loss_tar.item()})
    
    loss_all.backward()
    
    if loss_all.item() < best_loss:
        torch.save(model.state_dict(), os.path.join(log, model_name))
        best_loss = loss_all.item()

    print("Epoch: {}, Iter: {}, Loss: {}".format(epoch, count_itern_each_epoch, loss_all.item()), end="\t")   

    if itern % accum_iter == 0:
        optimizer.step()
        optimizer.zero_grad()

    count_itern_each_epoch += 1

loss,▅▅▁▆▅▃▃▆▅█▄▁
loss_src,█▅▂▅▅▅▅██▇▁▃
loss,2.21141
loss_src,5.60705
loss_tar,None.Tensor.item


KeyboardInterrupt: 